<a href="https://colab.research.google.com/github/sebavassou/SIG/blob/main/SIG_Enap_aula_08_analise_vetorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalar as bibliotecas de SIG
*Ambiente Linux*
1.   Atualiza o repositório local (sincroniza) e depois o sistema  - ***'apt'***
2.   Instala as dependências para as bibliotecas Python - ***'apt'***
3.   Atualiza o ***'pip'*** (instalado das bibliotecas python)
4.   Instala as bibliotecas de Python para SIG

Obs.:
- O Google Colab já possui alguma das bibliotecas
- Algumas bibliotecas depende de outras (instala automáticamente)

In [ ]:
%%bash
# Instalar as bibliotecas
apt update
apt upgrade
apt install libspatialindex-dev
apt install libgdal-dev libgdal-pcraster-dev libgdal-perl
apt install libsqlite3-mod-spatialite
apt install gdal-bin
pip -q install --upgrade pip
pip -q install shapely --no-binary shapely
pip -q install geopandas
pip -q install pycrs
pip -q install rasterio
pip -q install ipyleaflet
pip -q install localtileserver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,628 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,801 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,067 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,561 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.ne



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)






E: Unable to locate package libgdal-pcraster-dev




  DEPRECATION: Building 'pycrs' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pycrs'. Discussion can be found at https://github.com/pypa/pip/issues/6334


### Arquivos para Download
*   Base cartográfica 1:100 do Ibge do Estado de GO e do DF:  
https://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc100/go_df/versao2022/geopackage/bc100_godf_2022-11-14_gpkg.zip

In [ ]:
%%bash
wget -q https://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc100/go_df/versao2022/geopackage/bc100_godf_2022-11-14_gpkg.zip
unzip -q bc100_godf_2022-11-14_gpkg.zip
ls -lh *.gpkg

### Obter as informações das Camadas
*   Camadas?
    *    rod_trecho_rodoviario_l (rodovias)
    *    hid_trecho_drenagem_l (rios)
*   Identificação do SRC
*   Tipo de geometrias

In [ ]:
import geopandas as gpd
import shapely
import ipyleaflet
from localtileserver import TileClient, get_leaflet_tile_layer


def getMetadata(gdf):
    return {
        'layer_name': gdf.attrs['layer_name'],
        'epsg': gdf.crs.to_epsg(),
        'total': len(gdf),
        'geom_type_s': gdf.geom_type.unique()
    }


def printDic(data, indent=0):
    for k, v in data.items():
        prefix = " " * indent + f"- {k}: "
        if isinstance(v, dict):
            print( prefix)
            printDic(v, indent+1)
        else:
            print(f"{prefix}{v}")

#
filepath = 'bc100_godf_2022-11-14.gpkg'
#
layer = 'rod_trecho_rodoviario_l'
gdf_rod = gpd.read_file( filepath, layer=layer)
gdf_rod.attrs['layer_name'] = 'trecho_rodovias'
printDic( ( getMetadata( gdf_rod)  ) )
#
print()
#
layer = 'hid_trecho_drenagem_l'
gdf_rios = gpd.read_file( filepath, layer=layer)
gdf_rios.attrs['layer_name'] = 'trecho_rios'
printDic( getMetadata( gdf_rios ) )

- layer_name: trecho_rodovias
- epsg: 4674
- total: 50259
- geom_type_s: ['LineString']

- layer_name: trecho_rios
- epsg: 4674
- total: 208441
- geom_type_s: ['LineString' 'MultiLineString']


In [ ]:
# Rios
gdf_rios.geom_type.value_counts()

,count
LineString,208423
MultiLineString,18


In [ ]:
gdf_rios_corrigido = gdf_rios.explode(index_parts=False)
gdf_rios_corrigido.geom_type.value_counts()

,count
LineString,208462


### Projetar para EPSG 5880 (SIRGAS 2000 / Brazil Polyconic) as camadas
-    Para utilizar os operadores de distância é mais aprorpiado utlizar coordenadas planas

In [ ]:
gdf_rod_proj = gdf_rod.to_crs( 5880 )
gdf_rios_proj = gdf_rios_corrigido.to_crs( 5880 )

### Criar as nascentes a partir dos trechos de Rios

In [ ]:
def createSourceRiver(gdf, tolerance=5):
    gdf["ponto_inicial"] = gdf.geometry.apply(lambda geom: shapely.Point(geom.coords[0]))
    gdf["ponto_final"] = gdf.geometry.apply(lambda geom: shapely.Point(geom.coords[-1]))

    end_buff = [p.buffer(tolerance) for p in gdf["ponto_final"]]
    btree =  shapely.strtree.STRtree( end_buff )

    sources = []
    for p_ini in gdf["ponto_inicial"]:
        intersects = any( end_buff[buf].contains(p_ini) for buf in btree.query(p_ini) )
        if not intersects:
            sources.append(p_ini)

    return gpd.GeoDataFrame(geometry=sources, crs=gdf.crs)

gpd_source_rivers_proj = createSourceRiver( gdf_rios_proj )
gpd_source_rivers_proj.to_crs(4326).to_file('source_rivers.geojson', driver='GeoJSON')

### Locais (mais próximos) nas Rodovias Federais que estão a 200m das nascentes

In [ ]:
gdf_rod_fed_proj = gdf_rod_proj[ gdf_rod_proj['jurisdicao'] == 'Federal']

rod_coords = []
for p in gpd_source_rivers_proj.geometry:
    buff = p.buffer(200)
    gdf_ = gdf_rod_fed_proj[ gdf_rod_fed_proj.geometry.intersects(buff) ]
    if len(gdf_) > 0:
        line_near_id = gdf_.distance(p).idxmin()
        line = gdf_.loc[line_near_id].geometry
        point_interpolate = line.interpolate( line.project(p) )
        rod_coords.append( point_interpolate )
#
gdf_rod_coords_near_200_proj = gpd.GeoDataFrame(geometry=rod_coords, crs=gdf_rod_fed_proj.crs)
gdf_rod_coords_near_200_proj.to_crs(4326) .to_file('rod_coords_near_200.geojson', driver='GeoJSON')